In [1]:
import numpy as np
import pandas as pd

import matplotlib # Para ver la versión
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("madrid_sale_properties_raw_1.csv")

df


,url,property_native_id,price_eur,barrio,distrito,latitude,longitude,scrape_status,scraped_timestamp,description,...,Superficie solar,Superficie útil,Teléfono,Terraza,Tipo de casa,Tipo suelo,Trastero,Urbanizado,Vidrios dobles,page_source
0,https://www.pisos.com/comprar/piso-recoletos28...,4.756077e+10,9500000.0,Recoletos,Salamanca,"40,4221164","-3,6831486",Success,2025-05-28T17:29:28.477101,Exclusiva Propiedad en Recoletos junto al Parq...,...,NaN,499 m²,NaN,NaN,NaN,Tarima maciza,NaN,NaN,NaN,1
1,https://www.pisos.com/comprar/piso-universidad...,5.003730e+10,840000.0,Universidad-Malasaña,Centro,"40,422312","-3,706137",Success,2025-05-28T17:29:30.304660,UNICA INMOBILIARIA ofrece este impresionante p...,...,NaN,95 m²,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,https://www.pisos.com/comprar/piso-goya28009-4...,4.501932e+10,3600000.0,Goya,Salamanca,"40,4243638","-3,6705767",Success,2025-05-28T17:29:32.075647,UNICA INMOBILIARIA ofrece este impresionante i...,...,NaN,NaN,NaN,True,NaN,Tarima maciza,NaN,NaN,True,1
3,https://www.pisos.com/comprar/piso-opanel28019...,5.173469e+10,285000.0,Opañel,Carabanchel,40.3891,-3.7199,Success,2025-05-28T17:29:33.855489,Piso de 2 habitaciones de 60m² en madrid(28019...,...,NaN,60 m²,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,https://www.pisos.com/comprar/piso-rejas28022-...,5.422137e+10,539000.0,Rejas,San Blas,40.4420421,-3.5759197,Success,2025-05-28T17:29:35.657174,Espléndida vivienda exterior con una superfici...,...,NaN,101 m²,True,NaN,NaN,Tarima flotante,6 metros cuadrados,NaN,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3015,https://www.pisos.com/comprar/piso-puente_de_v...,5.256348e+10,279000.0,Portazgo,Puente de Vallecas,40.3837117,-3.6504517,Success,2025-05-28T19:00:00.644476,Agencia inmobiliaria de MADRID VALLECAS - Zona...,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,100
3016,https://www.pisos.com/comprar/piso-ciudad_line...,5.341673e+10,411000.0,Pueblo Nuevo,Ciudad Lineal,40.4263335,-3.6382045,Success,2025-05-28T19:00:02.401578,La Inmobiliaria Internacional CPM gestiona est...,...,NaN,120 m²,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100
3017,https://www.pisos.com/comprar/atico-valdemarin...,5.421518e+10,1550000.0,Valdemarín,Moncloa-Aravaca,40.4653434,-3.7629673,Success,2025-05-28T19:00:04.332734,"Lujo, vistas y confort a un paso de Madrid Des...",...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,100
3018,https://www.pisos.com/comprar/piso-trafalgar28...,5.338242e+10,2490000.0,Trafalgar,Chamberí,40.4355968,-3.7004786,Success,2025-05-28T19:00:06.100435,*********grupo inmara inmobiliaria************...,...,NaN,160 m²,NaN,NaN,NaN,NaN,True,NaN,NaN,100


In [2]:

# 1. Limpiar nombres de columnas: eliminar espacios y estandarizar todo en minúsculas
df.columns = (df.columns.str.strip().str.lower())

# 2. Reemplazar comas por puntos en todas las celdas de tipo string
df = df.applymap(lambda x: x.replace(',', '.') if isinstance(x, str) else x)

# 3. Función para limpiar y convertir superficies a float
def limpiar_superficie(col):
    return pd.to_numeric(
        df[col].astype(str).str.replace(' m²', '', regex=False).str.replace(',', '.'),
        errors='coerce'
    )

df['superficie construida'] = limpiar_superficie('superficie construida')
df['superficie útil'] = limpiar_superficie('superficie útil')

# 4. Modificar columna planta
def normalizar_planta(planta):
    if pd.isna(planta):
        return planta
    planta = str(planta).strip().lower()
    if "semisótano" in planta or "entresuelo" in planta or "sótano" in planta:
        return -1
    if "principal" in planta or "bajo" in planta:
        return 0
    if "más de 20" in planta:
        return 20
    planta = planta.replace("ª", "").replace("º", "")
    try:
        return int(planta)
    except ValueError:
        return planta  

df["planta"] = df["planta"].apply(normalizar_planta)

# 5. Intentar convertir todas las columnas a float (si aplica)
for col in df.columns:
    try:
        df[col] = pd.to_numeric(df[col], errors='ignore')
    except:
        pass  # Ignora columnas que no pueden convertirse

# 6. Eliminar filas duplicadas
df.drop_duplicates(inplace=True)

# 7. Eliminar columnas completamente vacías
df.dropna(axis=1, how='all', inplace=True)

# 8. Reiniciar índice ---
df.reset_index(drop=True, inplace=True)

df.head(4)


C:\Users\Marta\AppData\Local\Temp\ipykernel_18820\989738111.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace(',', '.') if isinstance(x, str) else x)
C:\Users\Marta\AppData\Local\Temp\ipykernel_18820\989738111.py:39: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


,url,property_native_id,price_eur,barrio,distrito,latitude,longitude,scrape_status,scraped_timestamp,description,...,superficie solar,superficie útil,teléfono,terraza,tipo de casa,tipo suelo,trastero,urbanizado,vidrios dobles,page_source
0,https://www.pisos.com/comprar/piso-recoletos28...,4.756077e+10,9500000.0,Recoletos,Salamanca,40.422116,-3.683149,Success,2025-05-28T17:29:28.477101,Exclusiva Propiedad en Recoletos junto al Parq...,...,NaN,499.0,NaN,NaN,NaN,Tarima maciza,NaN,NaN,NaN,1
1,https://www.pisos.com/comprar/piso-universidad...,5.003730e+10,840000.0,Universidad-Malasaña,Centro,40.422312,-3.706137,Success,2025-05-28T17:29:30.304660,UNICA INMOBILIARIA ofrece este impresionante p...,...,NaN,95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,https://www.pisos.com/comprar/piso-goya28009-4...,4.501932e+10,3600000.0,Goya,Salamanca,40.424364,-3.670577,Success,2025-05-28T17:29:32.075647,UNICA INMOBILIARIA ofrece este impresionante i...,...,NaN,NaN,NaN,True,NaN,Tarima maciza,NaN,NaN,True,1
3,https://www.pisos.com/comprar/piso-opanel28019...,5.173469e+10,285000.0,Opañel,Carabanchel,40.389100,-3.719900,Success,2025-05-28T17:29:33.855489,Piso de 2 habitaciones de 60m² en madrid(28019...,...,NaN,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [3]:
# Eliminar las columnas no relevantes

columns_to_drop = ['url', 'property_native_id', 'scrape_status', 'scraped_timestamp', 'page_source', 'energy_certificate_main_classification',
                   'referencia', 'description', 'luz', 'no se aceptan mascotas', 'urbanizado', 'calle asfaltada',
                    'calle alumbrada', 'tipo de casa', 'luz', 'se aceptan mascotas', 'superficie solar', 'teléfono',
                    'carpintería interior', 'interior', 'agua', 'soleado', 'carpintería exterior', 'tipo suelo',
                    'gastos de comunidad', 'gas', 'lavadero', 'orientación', 'armarios empotrados', 'comedor', 
                    'cocina equipada']

df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)


# Eliminar las filas de datos cuyo precio de venta es NaN
df = df.dropna(subset=["price_eur"])

# Rellenar NaN en superficie útil con el valor de superficie construida
df['superficie útil'] = df['superficie útil'].fillna(df['superficie construida'])

# Función para rellenar baños si es NaN
def rellenar_banos(row):
    if pd.isna(row['baños']):
        if not pd.isna(row['superficie construida']):
            return max(1, round(row['superficie construida'] / 70)) 
        else:
            return 1   # Al menos 1 baño
    else:
        return int(row['baños'])

df['baños'] = df.apply(rellenar_banos, axis=1)

import numpy as np

# Función para imputar habitaciones según superficie útil
def estimar_habitaciones(row):
    if pd.isna(row['habitaciones']):
        if not pd.isna(row['superficie útil']):
            sup = row['superficie útil']
            if sup < 50:
                return 1
            else:
                return int(np.floor(sup / 25))
        else:
            return np.nan  # Si tampoco hay superficie útil, no se puede estimar
    else:
        return row['habitaciones']

df['habitaciones'] = df.apply(estimar_habitaciones, axis=1)


# Sustituir NaN por 0 en las columnas indicadas
columnas_a_llenar_cero = ['adaptado a personas con movilidad reducida', 'planta']

for col in columnas_a_llenar_cero:
    if col in df.columns:
        df[col].fillna(0, inplace=True)


# Sustituir NaN por False en las columnas indicadas 
columnas_a_llenar_false = ['amueblado', 'energy_consumption_rating', 'energy_emissions_rating', 'aire acondicionado',
                           'calefacción', 'cocina equipada', 'garaje', 'exterior', 'trastero', 'ascensor', 'terraza',
                           'balcón', 'chimenea', 'piscina', 'jardín', 'vidrios dobles', 'sistema de seguridad',
                           'portero automático', 'puerta blindada']

for col in columnas_a_llenar_false:
    if col in df.columns:
        df[col].fillna(False, inplace=True)

# Modificar la columna conservación
df['conservación'] = df['conservación'].fillna('En buen estado')


C:\Users\Marta\AppData\Local\Temp\ipykernel_18820\3012149188.py:55: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(0, inplace=True)
C:\Users\Marta\AppData\Local\Temp\ipykernel_18820\3012149188.py:66: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doin

In [4]:
# Columnas con más del 80% de nulos
nulls = df.isnull().mean().sort_values(ascending=False)
print(nulls[nulls > 0.80])

Series([], dtype: float64)


In [5]:
# Se convierten en booleanas las siguientes columnas:

valores_terraza = {'2.60 m²', '25 m²', '2 terrazas', 'Balcones', '4',
       '2', '42', '2 terrazas acristaladas en la cocina y el salón',
       'Totalmente exterior y abierta', '200',
       'Alrededor de toda la vivienda', 'Metida en el salón', '33', '13',
       '7', '40', '30', 'Balcón y terraza', 'Dos terrazas', '6', '23',
       'Volada', '15 m²', '36.64', 'No acristalada',
       'Impresionante terraza de unos 50m²', 'Terraza', '3 terrazas',
       'Con jacuzzi', '3', 'Privada de 60 m²', '5 m²', '115m²',
       'Pequeña terraza', '8 metros acristalada por la cocina', 'mtrs=12',
       '5', '5 terazas total 10m',
       '22 m. y otra terraza ahora incluida en la vivienda', 'mtrs=0',
       '37', '70', 'Acristalada (con cámara térmica)', '106', '16',
       '10m²', '2 terras abiertas y una acristalada en cocina', '8.8',
       '2. una cerrada y una abierta', '20.85', 'En salón cerrada',
       'Pequeña', '110 m²', '130', '18', '11.98', 'Incorporada',
       '20. 40 y 50 m²', 'mtrs=6', '17', '12',
       '40 m². posibilidad de más', '14', '10', '53', '64 m²',
       'Con salida en un dormitorio', '11', '117 m²', '24', '56.45',
       'En el salón', '2 terrazas y solarium de 45.14 m².', '73 m²', '25',
       '20', '50', '43', '171', '5.5', '110.53', '119', '21',
       'Cerrada y con un pequeño trastero', 'A la calle.', '90', '45 m²',
       '120', '8', '17.11', '15', 'Vista espectaculares', 'De 20 y 10 n2',
       '6.00 m²', '6157', 'Acristalada', '22', '16m²',
       '8 m² y 5 m². 2 terrazas', '226', '60', '82', '35', '9', '63',
       'Balcon en cocina', 'mtrs=8'}
df['terraza'] = df['terraza'].isin(valores_terraza)

valores_piscina = {'Comunitaria', 'Propia', 'Privada', 'Gym y spa.', 'Con salorium'}
df['piscina'] = df['piscina'].isin(valores_piscina)

valores_trastero ={'6 metros cuadrados', 'Si grande', 'Doble',
       'Planta 6 nº 6', 'Hueco para realizar un trastero en el edificio',
       '11m²', '2 trasteros en la buhardilla', 'Tiene dos trasteros',
       '2 trasteros de', '6x4 m²', 'Incluido', '6 m²', '64',
       'Comparte un trastero', '4 metros', '31 m²', '9 m²'}
df['trastero'] = df['trastero'].isin(valores_trastero)

valores_calefacción = {'Central', 'Individual : gas natural', 'Gas natural',
       'Eléctrica', 'Individual', 'Climatizacion',
       'Suelo radiante', 'Individual - gas natural',
       'Calefacción individual', 'Gasoil', 'Con regundores', 'Aerotermia',
       'Calefaccion: suelo radiante',
       'Sistema de calefacción por suelo radiante',
       'Calefacción central con contador individual',
       'Suelo radiante de agua frio y calor. Geotermia cen',
       'Gas natural y biomasa', 'Bomba frio calor', 'Climatización',
       'Calefaccion', 'Termo electrico', 'Bomba frío-calor'}
df['calefacción'] = df['calefacción'].isin(valores_calefacción)

valores_ascensor = {'2 ascensores', 'Si 2', '2',
       'Es bajo pero la finca tiene', 'Recién instalado y pagado',
       'Nuevo', '3 ascensores', '2 principal y de servicio', '4'}
df['ascensor'] = df['ascensor'].isin(valores_ascensor)

valores_aire_acondicionado = {'Frío y calor', 'Frío', 'Individual',
       'Splits y unidades', 'A/a f/c', 'Comunitario', 'Aerotermia',
       'Frío-calor', 'Con bomba de calor', 'Centralizado',
       'A/a por conductos con sistema airzone', 'Tambien suelo radiante',
       'A/a f/c centralizado', 'En toda la casa', 'De ventana',
       'Por conductos independientes', 'Por conductos'}
df['aire acondicionado'] = df['aire acondicionado'].isin(valores_aire_acondicionado)

valores_no_amueblado = {'No', 'O sin mueble', 'También vacío'}
df['amueblado'] = df['amueblado'].apply(lambda x: False if pd.isna(x) or x in valores_no_amueblado else True)

valores_exterior = {'Todo exterior', 'Exterior planta calle',
       'Con muy buenas vistas', 'Exterior',
       'Si con dos balcones a la calle', 'Ladrillo',
       'Excelente iluminaci{on', 'Interior',
       'Salon y dormitorio principal', '6 balcones a la calle',
       'Es un esquinazo. Santa engracia y josé abascal',
       '5 balcones a la calle', 'Con terraza', 'Balcones', 'Con balcón',
       'Exterior calle', 'Acceso transitable amplio y ajardinado.',
       '2 balcones', 'Tres ventanales', '3 balcones a lagasca',
       'Toda la casa exterior', 'Un balcón y un mirador',
       '6 ventanas al exterior', 'Con balcones', 'A calle', '3 balcones',
       'Buenas vistas', '3 balcones y 1 mirador',
       'Todo exterior y muy soleado',
       'Salón . resto de loa vivienda a urbanización',
       'Muy luminoso y agradable'}
df['exterior'] = df['exterior'].isin(valores_exterior)

valores_garaje ={'1', '2', 'Más de 2'}
df['garaje'] = df['garaje'].isin(valores_garaje)

valores_jardín = {'Comunitario', 'Privado'}
df['jardín'] = df['jardín'].isin(valores_jardín)

valores_sistema_seguridad = {'Portero físico', 'Alarma', 'Camaras',
       'con cámaras de seguridad', 'vigilancia 24h', 'Conserje 24 h',
       'Puerta de seguridad', 'Sie', '2 turnos + cámaras nocturnas',
       'Cámaras de vigilancia', 'Cámaras t.v.', 'Conserje', '24h',
       'Servicio de portería', 'Garita para el control',
       'Portería. Portero físico', 'Portero', 'Conserje. Vigilancia 24h',
       'Portero fisico', 'Seguridad privada urbanizacion',
       'Vigilancia 24 horas', 'vigilancia 24h. con cámaras de seguridad'}
df['sistema de seguridad'] = df['sistema de seguridad'].isin(valores_sistema_seguridad)

valores_vidrios_dobles = {'Dobles de climalit',
       'Dobles de climalit. oscilovatiente. acusaticas', 'Doble climalit',
       'Climalit', 'Las ventanas son todas nuevas', 'Si de climalit',
       'Si. de climalit', 'Doble cámara . cuatro vidrios',
       'Aluminio climalit', 'Cerramientos de aluminio',
       'En todas las ventanas'}
df['vidrios dobles'] = df['vidrios dobles'].isin(valores_vidrios_dobles)

In [6]:
# Mostrar valores nulos ordenados (descendente)
df.isnull().sum().sort_values(ascending=False)

energy_consumption_value                      2327
energy_emissions_value                        2327
antigüedad                                    1311
superficie útil                                 10
superficie construida                           10
habitaciones                                     7
distrito                                         1
barrio                                           1
price_eur                                        0
jardín                                           0
piscina                                          0
planta                                           0
sistema de seguridad                             0
portero automático                               0
puerta blindada                                  0
exterior                                         0
terraza                                          0
trastero                                         0
garaje                                           0
calefacción                    

In [7]:
df

,price_eur,barrio,distrito,latitude,longitude,energy_consumption_rating,energy_consumption_value,energy_emissions_rating,energy_emissions_value,adaptado a personas con movilidad reducida,...,piscina,planta,portero automático,puerta blindada,sistema de seguridad,superficie construida,superficie útil,terraza,trastero,vidrios dobles
0,9500000.0,Recoletos,Salamanca,40.422116,-3.683149,False,NaN,False,NaN,0.0,...,False,1.0,Portero físico,False,False,499.0,499.0,False,False,False
1,840000.0,Universidad-Malasaña,Centro,40.422312,-3.706137,False,NaN,False,NaN,0.0,...,False,2.0,True,False,False,109.0,95.0,False,False,False
2,3600000.0,Goya,Salamanca,40.424364,-3.670577,False,NaN,False,NaN,0.0,...,False,3.0,True,True,False,379.0,379.0,False,False,False
3,285000.0,Opañel,Carabanchel,40.389100,-3.719900,E,151 kWh/m² año,D,31 Kg CO₂/m² año,0.0,...,False,0.0,False,False,False,60.0,60.0,False,False,False
4,539000.0,Rejas,San Blas,40.442042,-3.575920,E,152 kWh/m² año,D,30 Kg CO₂/m² año,1.0,...,True,2.0,True,True,False,127.0,101.0,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3015,279000.0,Portazgo,Puente de Vallecas,40.383712,-3.650452,E,211 kWh/m² año,E,44 Kg CO₂/m² año,0.0,...,False,2.0,False,False,False,113.0,113.0,False,False,False
3016,411000.0,Pueblo Nuevo,Ciudad Lineal,40.426333,-3.638205,False,NaN,False,NaN,0.0,...,False,0.0,False,False,False,130.0,120.0,False,False,False
3017,1550000.0,Valdemarín,Moncloa-Aravaca,40.465343,-3.762967,D,132 kWh/m² año,D,27 Kg CO₂/m² año,0.0,...,True,2.0,False,False,False,239.0,239.0,False,False,False
3018,2490000.0,Trafalgar,Chamberí,40.435597,-3.700479,False,NaN,False,NaN,0.0,...,False,4.0,False,True,False,185.0,160.0,False,False,False


In [8]:
df.to_csv("madrid_sale_properties_cleaned.csv", index=False)

In [ ]:
##############################################################################################################################